In [14]:
import os
import sys
current_directory = os.getcwd().replace('notebooks', '')
sys.path.append(current_directory)

import time
import polars as pl
from numpy import mean, std

In [5]:
from utils.helper import read_json
from utils.graph_reader import read_graph_instance, read_rakaj_graph_instance
from main.ilp_model import ilp_mwdds
from main.greedy_algorithm import greedy_mwds, greedy_mwdds, greedy_mwds_aco
from main.genetic_algorithm import GeneticAlgorithm, fitness_func
from utils.graph import Graph

test_instances_pth = '../instances/test_instances/'
test_instances = os.listdir(test_instances_pth)
test_instances = [test_instances_pth + instance for instance in test_instances]

instances_s_pth = '../instances/small/'
instances_m_pth = '../instances/medium/'
instances_l_pth = '../instances/large/'
instances_s_dict = read_json('../instances/small/small_instances.json')
instances_m_dict = read_json('../instances/medium/medium_instances.json')
instances_l_dict = read_json('../instances/large/large_instances.json')

In [6]:
def save_results_to_csv(results, output_csv):
    df = pl.DataFrame({
        'instance': results['instance'],
        'ga_mean': results['ga_mean'],
        'ga_max': results['ga_max'],
        'ga_std': results['ga_std'],
        'ga_time': results['ga_time'],
        'greedy': results['greedy'],
        'greedy_time': results['greedy_time'],
        'ilp': results['ilp'],
        'ilp_time': results['ilp_time']
    })

    df.write_csv(output_csv)

In [16]:
results = {
    'instance': [],
    'ga_mean': [],
    'ga_max': [],
    'ga_std': [],
    'ga_time': [],
    'greedy': [],
    'greedy_time': [],
    'ilp': [],
    'ilp_time': []
}

params = {
    "n_generations":100,  "population_size":100, "selection_method":'tournament', "crossover_probability":0.9, "mutation_probability":0.01, "mutation_increase_factor":1.5, "inc_mutation":20
}

for instances in test_instances:
        vertices_w, edges = read_graph_instance(instances)

        results['instance'].append("test")

        ga_results = []
        ga_time = []
        graph = Graph()
        graph.create_from_data(vertices_w, edges)
        for i in range(10):
            gen_algo = GeneticAlgorithm(graph=graph, fitness_fn=fitness_func, max_time=600, max_no_improvement=100, **params)
            gen_algo.run()
            ga_results.append(gen_algo.previous_best_fitness)
            ga_time.append(gen_algo.running_time)
    
        results['ga_mean'].append(round(mean(ga_results),6))
        results['ga_max'].append(max(ga_results))
        results['ga_std'].append(round(std(ga_results), 6))
        results['ga_time'].append(round(mean(ga_time), 4))

        start = time.time()
        greedy_ds, greedy_w = greedy_mwdds(vertices_w, edges)
        results['greedy'].append(greedy_w)
        results['greedy_time'].append(round(time.time() - start, 4))
        
        start = time.time()
        ilp_ds, ilp_w = ilp_mwdds(vertices_w, edges)
        results['ilp'].append(ilp_w)
        results['ilp_time'].append(round(time.time() - start, 4))

# save file with params
save_results_to_csv(results, f'../results/new_test_instances_g{params["n_generations"]}_p{params["population_size"]}.csv')

|                                        | ▁▃▅ 0/10 [0%] in 0s (~0s, 0.0/s) 

|████⚠︎                                   | (!) 1/10 [10%] in 1.0s (0.99/s)      
|████⚠︎                                   | (!) 1/10 [10%] in 0.9s (1.11/s)      
|████⚠︎                                   | (!) 1/10 [10%] in 0.9s (1.09/s)      
|████⚠︎                                   | (!) 1/10 [10%] in 1.1s (0.88/s)      
|████⚠︎                                   | (!) 1/10 [10%] in 0.8s (1.31/s)      


In [6]:
results = {
    'instance': [],
    'ga_mean': [],
    'ga_max': [],
    'ga_std': [],
    'ga_time': [],
    'greedy': [],
    'greedy_time': [],
    'ilp': [],
    'ilp_time': []
}

params = {
    "n_generations":500,  "population_size":1000, "selection_method":'tournament', "crossover_probability":0.9, "mutation_probability":0.01, "mutation_increase_factor":1.5, "inc_mutation":100
}

for folder, instances in instances_s_dict.items():
        for i in instances:
            instance_pth = os.path.join(instances_s_pth, folder, i)
            vertices_w, edges = read_rakaj_graph_instance(instance_pth)

            results['instance'].append(folder)

            ga_results = []
            ga_time = []
            graph = Graph()
            graph.create_from_data(vertices_w, edges)
            for i in range(10):
                gen_algo = GeneticAlgorithm(graph=graph, fitness_fn=fitness_func, max_time=600, max_no_improvement=250, **params)
                gen_algo.run()
                ga_results.append(gen_algo.previous_best_fitness)
                ga_time.append(gen_algo.running_time)
                # print(f"{folder} {i}, {gen_algo.previous_best_fitness}")

            results['ga_mean'].append(round(mean(ga_results),6))
            results['ga_max'].append(max(ga_results))
            results['ga_std'].append(round(std(ga_results), 6))
            results['ga_time'].append(round(mean(ga_time), 4))

            # start = time.time()
            # greedy_ds, greedy_w = greedy_mwdds(vertices_w, edges)
            # results['greedy'].append(greedy_w)
            # results['greedy_time'].append(round(time.time() - start, 4))
            
            # start = time.time()
            # ilp_ds, ilp_w = ilp_mwdds(vertices_w, edges)
            # results['ilp'].append(ilp_w)
            # results['ilp_time'].append(round(time.time() - start, 4))
            results['greedy'].append(0)
            results['greedy_time'].append(0)
            results['ilp'].append(0)
            results['ilp_time'].append(0)


save_results_to_csv(results, f'../results/new_small_instances_g{params["n_generations"]}_p{params["population_size"]}.csv')

In [7]:
results = {
    'instance': [],
    'ga_mean': [],
    'ga_max': [],
    'ga_std': [],
    'ga_time': [],
    'greedy': [],
    'greedy_time': [],
    'ilp': [],
    'ilp_time': []
}

params = {
    "n_generations":500,  "population_size":1000, "selection_method":'tournament', "crossover_probability":0.9, "mutation_probability":0.01, "mutation_increase_factor":1.5, "inc_mutation":100
}

for folder, instances in instances_m_dict.items():
        for i in instances:
            instance_pth = os.path.join(instances_m_pth, folder, i)
            vertices_w, edges = read_rakaj_graph_instance(instance_pth)
            
            results['instance'].append(folder)

            ga_results = []
            ga_time = []
            graph = Graph()
            graph.create_from_data(vertices_w, edges)
            for i in range(10):
                gen_algo = GeneticAlgorithm(graph=graph, fitness_fn=fitness_func, max_time=600, max_no_improvement=200, **params)
                gen_algo.run()
                ga_results.append(gen_algo.previous_best_fitness)
                ga_time.append(gen_algo.running_time)
                print(f"{folder} {i}, {ga_results}")
            
            results['ga_mean'].append(round(mean(ga_results), 6))
            results['ga_max'].append(max(ga_results))
            results['ga_std'].append(round(std(ga_results), 6))
            results['ga_time'].append(round(mean(ga_time), 4))

            # start = time.time()
            # greedy_ds, greedy_w = greedy_mwdds(vertices_w, edges)
            # results['greedy'].append(greedy_w)
            # results['greedy_time'].append(round(time.time() - start, 4))
            
            # start = time.time()
            # ilp_ds, ilp_w = ilp_mwdds(vertices_w, edges)
            # results['ilp'].append(ilp_w)
            # results['ilp_time'].append(round(time.time() - start, 4))
            results['greedy'].append(0)
            results['greedy_time'].append(0)
            results['ilp'].append(0)
            results['ilp_time'].append(0)

save_results_to_csv(results, f'../results/new_mid_instances_g{params["n_generations"]}_p{params["population_size"]}.csv')

In [9]:
results = {
    'instance': [],
    'ga_mean': [],
    'ga_max': [],
    'ga_std': [],
    'ga_time': [],
    'greedy': [],
    'greedy_time': [],
    'ilp': [],
    'ilp_time': []
}

params = {
    "n_generations":1000,  "population_size":1000, "selection_method":'tournament', "crossover_probability":0.9, "mutation_probability":0.01, "mutation_increase_factor":1.5, "inc_mutation":100
}

for folder, instances in instances_l_dict.items():
        for i in instances:
            instance_pth = os.path.join(instances_l_pth, folder, i)
            vertices_w, edges = read_rakaj_graph_instance(instance_pth)

            results['instance'].append(folder)

            ga_results = []
            ga_time = []
            graph = Graph()
            graph.create_from_data(vertices_w, edges)
            for i in range(10):
                gen_algo = GeneticAlgorithm(graph=graph, fitness_fn=fitness_func, max_time=600, max_no_improvement=200, **params)
                gen_algo.run()
                ga_results.append(gen_algo.previous_best_fitness)
                ga_time.append(gen_algo.running_time)
                print(f"{folder} {i}, {ga_results}")

            results['ga_mean'].append(round(mean(ga_results),6))
            results['ga_max'].append(max(ga_results))
            results['ga_std'].append(round(std(ga_results), 6))
            results['ga_time'].append(round(mean(ga_time), 4))

            # start = time.time()
            # greedy_ds, greedy_w = greedy_mwdds(vertices_w, edges)
            # results['greedy'].append(greedy_w)
            # results['greedy_time'].append(round(time.time() - start, 4))
            
            # start = time.time()
            # ilp_ds, ilp_w = ilp_mwdds(vertices_w, edges)
            # results['ilp'].append(ilp_w)
            # results['ilp_time'].append(round(time.time() - start, 4))
            results['greedy'].append(0)
            results['greedy_time'].append(0)
            results['ilp'].append(0)
            results['ilp_time'].append(0)

save_results_to_csv(results, f'../results/new_large_instances_g{params["n_generations"]}_p{params["population_size"]}.csv')

V500E1000 0, [10145]
V500E1000 1, [10145, 9531]
V500E1000 2, [10145, 9531, 9734]
V500E1000 3, [10145, 9531, 9734, 9334]
V500E1000 4, [10145, 9531, 9734, 9334, 9427]
V500E1000 5, [10145, 9531, 9734, 9334, 9427, 9757]
V500E1000 6, [10145, 9531, 9734, 9334, 9427, 9757, 9791]
V500E1000 7, [10145, 9531, 9734, 9334, 9427, 9757, 9791, 9518]
V500E1000 8, [10145, 9531, 9734, 9334, 9427, 9757, 9791, 9518, 9571]
V500E1000 9, [10145, 9531, 9734, 9334, 9427, 9757, 9791, 9518, 9571, 9739]
V500E2000 0, [6338]
V500E2000 1, [6338, 6305]
V500E2000 2, [6338, 6305, 6851]
V500E2000 3, [6338, 6305, 6851, 6982]
V500E2000 4, [6338, 6305, 6851, 6982, 6854]
V500E2000 5, [6338, 6305, 6851, 6982, 6854, 6696]
V500E2000 6, [6338, 6305, 6851, 6982, 6854, 6696, 7446]
V500E2000 7, [6338, 6305, 6851, 6982, 6854, 6696, 7446, 7081]
V500E2000 8, [6338, 6305, 6851, 6982, 6854, 6696, 7446, 7081, 6967]
V500E2000 9, [6338, 6305, 6851, 6982, 6854, 6696, 7446, 7081, 6967, 6851]
V500E5000 0, [3957]
V500E5000 1, [3957, 4039]
V500